In [ ]:
import tess_cpm
import numpy as np
import matplotlib.pyplot as plt
import lightkurve as lk
from astropy.io import fits
from astropy.wcs import WCS

In [ ]:
plt.rcParams["figure.figsize"] = (12, 10)

In [ ]:
fits_file = "supernovae/tess-s0001-4-1_64.525833_-63.615669_100x100_astrocut.fits"

In [ ]:
sn = tess_cpm.Source(fits_file, remove_bad=True)

In [ ]:
sn.set_aperture(rowrange=[49, 52], colrange=[49, 52])
sn.plot_cutout(rowrange=[45, 55], colrange=[45, 55], show_aperture=True)

In [ ]:
sn.plot_pix_by_pix(data_type="normalized_flux")

In [ ]:
sn.add_cpm_model()
# sn.set_regs([0.01])
# sn.holdout_fit_predict(k=2);

In [ ]:
# sn.plot_pix_by_pix(data_type="rescaled_cpm_subtracted_lc", split=True)

In [ ]:
# aperture_lc = sn.get_aperture_lc(data_type="cpm_subtracted_lc")
# cpm_prediction = sn.get_aperture_lc(data_type="cpm_prediction")

In [ ]:
# plt.plot(sn.time, aperture_lc)
# plt.plot(sn.time, cpm_prediction)

In [ ]:
sn.add_poly_model()
sn.set_regs([0.01, 1])
sn.holdout_fit_predict(k=20);

In [ ]:
sn.plot_pix_by_pix(data_type="rescaled_cpm_subtracted_lc", split=True)
# sn.plot_pix_by_pix(data_type="cpm_prediction")
# sn.plot_pix_by_pix(data_type="poly_model_prediction")

In [ ]:
flux = sn.get_aperture_lc(data_type="normalized_flux")
cpm_prediction = sn.get_aperture_lc(data_type="cpm_prediction")
# poly_prediction = sn.get_aperture_lc(data_type="poly_model_prediction")
apt_lc = sn.get_aperture_lc(data_type="rescaled_cpm_subtracted_lc")

In [ ]:
# plt.plot(sn.time, flux)
# plt.plot(sn.time, cpm_prediction)
# plt.plot(sn.time, poly_prediction)
plt.plot(sn.time, apt_lc)

In [ ]:
sn.lsq(0.1, rescale=True, polynomials=True)

In [ ]:
tess_cpm.summary_plot(sn, 20, subtract_polynomials=False, save=False)

In [ ]:
sn.get_aperture_lc(show_pixel_lc=True, show_aperture_lc=True)

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(sn.time, sn.rescaled_target_fluxes, '.')
n = 10
# locs, mask = c.get_contributing_pixels(n)
# print(c.rescaled_predictor_pixels_fluxes.shape)
# print(locs[2])
# plt.plot(c.time, c.rescaled_im_fluxes[:, locs[0][0], locs[0][1]], '-')
# plt.plot(c.time, c.rescaled_predictor_pixels_fluxes[:, 4])
for i in range(n):
    plt.figure(figsize=(12, 8))
    t = sn.rescaled_target_fluxes
    p = sn.rescaled_predictor_pixels_fluxes[:, i]
    plt.plot(sn.time, t, '.')
    plt.plot(sn.time, p, '-', alpha=0.4)
    plt.title(np.dot(p, t)/(np.linalg.norm(p)*np.linalg.norm(t)), fontsize=25)
    plt.show();

In [ ]:
fits_file = "supernovae/tess-s0002-4-1_64.525833_-63.615669_100x100_astrocut.fits"

In [ ]:
sector2 = tess_cpm.CPM(fits_file, remove_bad=False)

In [ ]:
sector2.set_poly_model(2, 4, 0.5)
sector2.set_target(50, 50)
sector2.set_exclusion(10)
# sector2.set_predictor_pixels(256)
sector2.set_predictor_pixels(256, method='cosine_similarity')

In [ ]:
sector2.lsq(0.1, rescale=True, polynomials=True)

In [ ]:
tess_cpm.summary_plot(sector2, 20, subtract_polynomials=False, save=False)

In [ ]:
plt.rcParams['figure.figsize'] = (12, 8)
prediction, residual = sn.xval(0.1)

In [ ]:
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
s = tess_cpm.Source(fits_file)
pm = tess_cpm.PixelModel(s.target_data, 50, 50)

In [ ]:
pm.add_cpm_model()
pm.add_poly_model()
pm.set_regs([1.0, 5.0])

In [ ]:
print(pm.time.size)
times, predictions = pm.holdout_predictions(k=3)
for time, prediction in zip(times, predictions):
    plt.plot(time, prediction)

In [ ]:
p = np.concatenate(predictions)
# plt.plot(pm.time, p)
plt.plot(pm.time, pm.y-p)

In [ ]:
pm.fit(save=True)
prediction = pm.predict()
cpm_prediction = pm.cpm.predict()

In [ ]:
plt.plot(pm.time, pm.y - cpm_prediction, ".")
# plt.plot(pm.time, pm.poly_model.predict(), '.')